# Downloading dataset and Unzip 

In [2]:
#!unzip anime-dataset-2023.csv1.zip

Archive:  anime-dataset-2023.csv1.zip
  inflating: anime-dataset-2023.csv  


In [15]:
import pickle
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sentence_transformers import SentenceTransformer

/home/shigin/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data processing

In [2]:
df = pd.read_csv('anime-dataset-2023.csv')

In [3]:
df.head()

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",...,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",...,Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,1795,613,42829.0,111931,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",...,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,5126,14,6413.0,15001,https://cdn.myanimelist.net/images/anime/7/215...


In [4]:
# df.dropna()

In [5]:
df.drop(['anime_id','Other name','Score','Premiered','Producers', 'Licensors','Rank', 'Popularity', 'Favorites', 'Scored By', 'Members'],axis = 1,inplace = True)

In [6]:
df

,Name,English name,Genres,Synopsis,Type,Episodes,Aired,Status,Studios,Source,Duration,Rating,Image URL
0,Cowboy Bebop,Cowboy Bebop,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",Finished Airing,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),https://cdn.myanimelist.net/images/anime/4/196...
1,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",Finished Airing,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),https://cdn.myanimelist.net/images/anime/1439/...
2,Trigun,Trigun,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",Finished Airing,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,https://cdn.myanimelist.net/images/anime/7/203...
3,Witch Hunter Robin,Witch Hunter Robin,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",Finished Airing,Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,https://cdn.myanimelist.net/images/anime/10/19...
4,Bouken Ou Beet,Beet the Vandel Buster,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",Finished Airing,Toei Animation,Manga,23 min per ep,PG - Children,https://cdn.myanimelist.net/images/anime/7/215...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24900,Wu Nao Monu,UNKNOWN,"Comedy, Fantasy, Slice of Life",No description available for this anime.,ONA,15.0,"Jul 4, 2023 to ?",Not yet aired,UNKNOWN,Web manga,Unknown,PG-13 - Teens 13 or older,https://cdn.myanimelist.net/images/anime/1386/...
24901,Bu Xing Si: Yuan Qi,Blader Soul,"Action, Adventure, Fantasy",No description available for this anime.,ONA,18.0,"Jul 27, 2023 to ?",Not yet aired,UNKNOWN,Web novel,Unknown,PG-13 - Teens 13 or older,https://cdn.myanimelist.net/images/anime/1383/...
24902,Di Yi Xulie,The First Order,"Action, Adventure, Fantasy, Sci-Fi",No description available for this anime.,ONA,16.0,"Jul 19, 2023 to ?",Finished Airing,UNKNOWN,Web novel,Unknown,PG-13 - Teens 13 or older,https://cdn.myanimelist.net/images/anime/1130/...
24903,Bokura no Saishuu Sensou,UNKNOWN,UNKNOWN,A music video for the song Bokura no Saishuu S...,Music,1.0,"Apr 23, 2022",Finished Airing,UNKNOWN,Original,3 min,PG-13 - Teens 13 or older,https://cdn.myanimelist.net/images/anime/1931/...


In [7]:
df['Genres']= df['Genres'].str.replace(', ',' ')

In [8]:
df['Genres']

0              Action Award Winning Sci-Fi
1                            Action Sci-Fi
2                  Action Adventure Sci-Fi
3        Action Drama Mystery Supernatural
4           Adventure Fantasy Supernatural
                       ...                
24900         Comedy Fantasy Slice of Life
24901             Action Adventure Fantasy
24902      Action Adventure Fantasy Sci-Fi
24903                              UNKNOWN
24904                              UNKNOWN
Name: Genres, Length: 24905, dtype: object

In [9]:
df.drop(df[(df['Genres'] == 'UNKNOWN') & (df['Synopsis'] == 'No description available for this anime.')].index, inplace=True)

In [10]:
df['tags'] = df.apply(lambda row: row['Synopsis'] if row['Genres'] == 'UNKNOWN' else row['Synopsis'] + ' ' + row['Genres'], axis=1)


In [11]:
df

,Name,English name,Genres,Synopsis,Type,Episodes,Aired,Status,Studios,Source,Duration,Rating,Image URL,tags
0,Cowboy Bebop,Cowboy Bebop,Action Award Winning Sci-Fi,"Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",Finished Airing,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),https://cdn.myanimelist.net/images/anime/4/196...,"Crime is timeless. By the year 2071, humanity ..."
1,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,Action Sci-Fi,"Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",Finished Airing,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),https://cdn.myanimelist.net/images/anime/1439/...,"Another day, another bounty—such is the life o..."
2,Trigun,Trigun,Action Adventure Sci-Fi,"Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",Finished Airing,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,https://cdn.myanimelist.net/images/anime/7/203...,"Vash the Stampede is the man with a $$60,000,0..."
3,Witch Hunter Robin,Witch Hunter Robin,Action Drama Mystery Supernatural,Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",Finished Airing,Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,https://cdn.myanimelist.net/images/anime/10/19...,Robin Sena is a powerful craft user drafted in...
4,Bouken Ou Beet,Beet the Vandel Buster,Adventure Fantasy Supernatural,It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",Finished Airing,Toei Animation,Manga,23 min per ep,PG - Children,https://cdn.myanimelist.net/images/anime/7/215...,It is the dark century and the people are suff...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24900,Wu Nao Monu,UNKNOWN,Comedy Fantasy Slice of Life,No description available for this anime.,ONA,15.0,"Jul 4, 2023 to ?",Not yet aired,UNKNOWN,Web manga,Unknown,PG-13 - Teens 13 or older,https://cdn.myanimelist.net/images/anime/1386/...,No description available for this anime. Comed...
24901,Bu Xing Si: Yuan Qi,Blader Soul,Action Adventure Fantasy,No description available for this anime.,ONA,18.0,"Jul 27, 2023 to ?",Not yet aired,UNKNOWN,Web novel,Unknown,PG-13 - Teens 13 or older,https://cdn.myanimelist.net/images/anime/1383/...,No description available for this anime. Actio...
24902,Di Yi Xulie,The First Order,Action Adventure Fantasy Sci-Fi,No description available for this anime.,ONA,16.0,"Jul 19, 2023 to ?",Finished Airing,UNKNOWN,Web novel,Unknown,PG-13 - Teens 13 or older,https://cdn.myanimelist.net/images/anime/1130/...,No description available for this anime. Actio...
24903,Bokura no Saishuu Sensou,UNKNOWN,UNKNOWN,A music video for the song Bokura no Saishuu S...,Music,1.0,"Apr 23, 2022",Finished Airing,UNKNOWN,Original,3 min,PG-13 - Teens 13 or older,https://cdn.myanimelist.net/images/anime/1931/...,A music video for the song Bokura no Saishuu S...


In [13]:
!pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable


In [16]:
# Generate embeddings using a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(df['tags'].tolist(), show_progress_bar=True)

Batches: 100%|████████████████████████████████| 727/727 [09:31<00:00,  1.27it/s]


In [17]:
# Try different number of neighbors
neighbors_to_try = [5, 10, 15, 20]

# Try different metrics
metrics_to_try = ['cosine', 'euclidean', 'manhattan']

results = []

for metric in metrics_to_try:
    for n in neighbors_to_try:
        print(f"Training KNN with {n} neighbors and {metric} metric")
        knn = NearestNeighbors(n_neighbors=n, metric=metric)
        knn.fit(embeddings)
        
        # Pick a random anime to test
        random_idx = 100  # or you can randomize
        distances, indices = knn.kneighbors([embeddings[random_idx]])
        
        avg_distance = distances[0][1:].mean()  # skip self
        results.append((metric, n, avg_distance))

# Print results sorted by average distance (lower is better for cosine, higher for euclidean)
results = sorted(results, key=lambda x: x[2])
for r in results:
    print(f"Metric: {r[0]}, Neighbors: {r[1]}, Avg Distance: {r[2]}")


Training KNN with 5 neighbors and cosine metric
Training KNN with 10 neighbors and cosine metric
Training KNN with 15 neighbors and cosine metric
Training KNN with 20 neighbors and cosine metric
Training KNN with 5 neighbors and euclidean metric
Training KNN with 10 neighbors and euclidean metric
Training KNN with 15 neighbors and euclidean metric
Training KNN with 20 neighbors and euclidean metric
Training KNN with 5 neighbors and manhattan metric
Training KNN with 10 neighbors and manhattan metric
Training KNN with 15 neighbors and manhattan metric
Training KNN with 20 neighbors and manhattan metric
Metric: cosine, Neighbors: 5, Avg Distance: 0.39442622661590576
Metric: cosine, Neighbors: 10, Avg Distance: 0.4806709885597229
Metric: cosine, Neighbors: 15, Avg Distance: 0.5141919255256653
Metric: cosine, Neighbors: 20, Avg Distance: 0.5338855981826782
Metric: euclidean, Neighbors: 5, Avg Distance: 0.8792427778244019
Metric: euclidean, Neighbors: 10, Avg Distance: 0.9732407993740506
Me

In [18]:
# Train the KNN model
knn = NearestNeighbors(n_neighbors=11, metric='cosine')
knn.fit(embeddings)

# Save the trained KNN model and embeddings
with open('anime_knn_model.pkl', 'wb') as f:
    pickle.dump(knn, f)

with open('anime_embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings, f)

# Save the processed DataFrame
df.to_csv('anime_data.csv', index=False)